In [99]:
import pandas as pd
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from xgboost import XGBClassifier

In [100]:
base_treino = pd.read_csv(r"D:\pipeline_dados\data_raw\base_orcamentos_treino.csv")
x_teste_df = pd.read_csv(r"D:\pipeline_dados\base_orcamentos_nov.csv")

In [107]:
x_teste_df

,dias_desejo,dias_em_casa,ds_tipo_obra,nm_prop_oportunidade,proprietario,promocional,tipo_loja,banheiro,toda_casa,cozinha,...,amb_outros,revestimento,officina,loucas_metais,arg_rejunte,categ_outros,vl_frete,vlr_orcamento,convertidos,nao_convertidos
0,4,23,Reforma,fabiano martins werutsky,zzzzzzzXzzzzzzzXzzzzzzzz,S,Tradicional A,S,N,N,...,S,1,0,0,1,0,63.33,823.62,343,438
1,7,23,Reforma,rafaela de sousa paonessa loureiro,zzzzzzzXzzXzzzzzXzzzzzzzzXzzzzzzzz,S,Tradicional A,N,S,N,...,S,2,0,0,0,0,83.33,1506.96,342,406
2,1,23,Reforma,rafaela de sousa paonessa loureiro,zzzzzzzXzzXzzzzzXzzzzzzzzXzzzzzzzz,S,Tradicional A,N,S,N,...,S,0,0,0,1,0,0.00,18.90,342,406
3,29,23,NaN,juliana westphalen gonçalves,zzzzzzzXzzzzzzzzzzXzzzzzzzzz,S,Tradicional A,N,S,N,...,S,0,1,0,3,0,106.67,20094.65,488,537
4,1,23,Reforma,rafaela de sousa paonessa loureiro,zzzzzzzXzzXzzzzzXzzzzzzzzXzzzzzzzz,S,Tradicional A,N,S,N,...,S,0,0,0,2,0,16.67,57.80,342,406
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,1,0,Reforma,fabiano martins werutsky,zzzzzzzXzzzzzzzXzzzzzzzz,S,Tradicional A,N,N,N,...,S,1,0,0,2,0,56.67,199.90,343,438
236,7,0,Nova Obra,fabiano martins werutsky,zzzzzzzXzzzzzzzXzzzzzzzz,S,Tradicional A,S,N,N,...,N,6,0,6,0,0,0.00,4106.58,343,438
237,6,0,Reforma,matheus homrich,zzzzzzzXzzzzzzz,S,Tradicional A,N,N,N,...,S,1,0,0,4,0,215.00,3724.22,184,197
238,39,0,Reforma,juliana westphalen gonçalves,zzzzzzzXzzzzzzzzzzXzzzzzzzzz,S,Tradicional A,N,N,N,...,S,1,0,0,4,0,120.00,5788.80,488,537


In [108]:
base_treino

,id_quote,dias_desejo,dias_em_casa,ds_tipo_obra,nm_prop_oportunidade,proprietario,promocional,convertido,tipo_loja,banheiro,...,amb_outros,revestimento,officina,loucas_metais,arg_rejunte,categ_outros,vl_frete,vlr_orcamento,convertidos,nao_convertidos
0,0Q03s000000GRoxCAG,203,534,Nova Obra,leticia silva de arruda,zzzzzzzXzzzzzXzzXzzzzzz,S,N,Tradicional A,S,...,S,7,2,0,8,0,153.33,76444.94,412,330
1,0Q03s000000GRsGCAW,11,534,Nova Obra,leticia silva de arruda,zzzzzzzXzzzzzXzzXzzzzzz,S,S,Tradicional A,N,...,N,1,0,0,0,0,26.67,64.80,412,330
2,0Q03s000000GRwXCAW,15,534,Nova Obra,fabiano martins werutsky,zzzzzzzXzzzzzzzXzzzzzzzz,S,N,Tradicional A,N,...,S,1,0,0,0,0,266.67,14763.62,343,438
3,0Q03s000000GS0UCAW,30,534,Nova Obra,leticia silva de arruda,zzzzzzzXzzzzzXzzXzzzzzz,S,S,Tradicional A,N,...,S,1,0,0,4,0,66.67,1301.09,412,330
4,0Q03s000000GS1cCAG,29,534,Reforma,rafaela de sousa paonessa loureiro,zzzzzzzXzzXzzzzzXzzzzzzzzXzzzzzzzz,S,N,Tradicional A,N,...,S,0,0,10,0,0,83.33,3276.30,342,406
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5592,0Q03s000001mURcCAM,24,39,Nova Obra,fabiano martins werutsky,zzzzzzzXzzzzzzzXzzzzzzzz,S,N,Tradicional A,N,...,S,1,0,0,4,0,93.33,1905.18,343,438
5593,0Q03s000001mUX1CAM,1,39,Reforma,patricia agnes,zzzzzzzzXzzzzz,S,S,Tradicional A,N,...,S,1,0,0,0,0,60.00,131.27,348,503
5594,0Q03s000001mUdiCAE,52,39,Nova Obra,juliana westphalen gonçalves,zzzzzzzXzzzzzzzzzzXzzzzzzzzz,S,N,Tradicional A,S,...,S,8,0,0,10,0,493.33,35609.59,488,538
5595,0Q03s000001mUh6CAE,196,39,Reforma,matheus homrich,zzzzzzzXzzzzzzz,S,S,Tradicional A,S,...,S,7,4,0,18,0,576.45,27856.07,184,197


In [101]:
y_teste = x_teste_df['convertido']
x_teste_df = x_teste_df.drop(['convertido', 'id_quote'], axis=1)

categoricos = ['ds_tipo_obra', 'nm_prop_oportunidade', 'proprietario', 'promocional', 'tipo_loja',
               'banheiro', 'toda_casa', 'cozinha', 'area_externa', 'itens_assentamento', 'sala',
               'piscina', 'area_servico', 'garagem', 'dormitorio', 'varanda', 'escada', 'amb_outros']

date_column = 'dt_criacao'

if date_column in base_treino.columns:
    print(f"Removing date column: {date_column}")
    base_treino = base_treino.drop(date_column, axis=1, errors='ignore')
    if date_column in categoricos:
        categoricos.remove(date_column)
else:
    print(f"Date column '{date_column}' not found in the training data.")

if date_column in x_teste_df.columns:
    x_teste_df = x_teste_df.drop(date_column, axis=1, errors='ignore')
    if date_column in categoricos:
        categoricos.remove(date_column)
else:
    print(f"Date column '{date_column}' not found in the test data.")

label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

encoder = OneHotEncoder(drop='if_binary', handle_unknown='ignore')
X_encoded = pd.DataFrame(encoder.fit_transform(X[categoricos]).toarray(),
                          columns=encoder.get_feature_names_out(categoricos))

X_final = pd.concat([X.drop(categoricos, axis=1), X_encoded], axis=1)

X_test_encoded = pd.DataFrame(encoder.transform(x_teste_df[categoricos]).toarray(),
                               columns=encoder.get_feature_names_out(categoricos))
X_test_final = pd.concat([x_teste_df.drop('id_quote', axis=1, errors='ignore'), X_test_encoded], axis=1).drop(categoricos, axis=1).fillna(0)

X = X.select_dtypes(exclude=['datetime64[ns]']) 


X_train, X_valid, y_train, y_valid = train_test_split(X_final, y, test_size=0.2, random_state=42)

xgb_model = XGBClassifier()

xgb_model.fit(X_train, y_train)

y_pred = xgb_model.predict(X_valid)

accuracy = accuracy_score(y_valid, y_pred)
print(f"Validation Accuracy: {accuracy}")

y_test_pred = xgb_model.predict(X_test_final)


print(y_test_pred)

Removing date column: dt_criacao
Validation Accuracy: 0.7035714285714286


d:\spark_testes\venv\lib\site-packages\sklearn\preprocessing\_encoders.py:228: UserWarning: Found unknown categories in columns [1, 2] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


[0 1 1 0 1 1 1 0 1 0 0 0 0 0 0 0 1 1 0 1 0 1 1 0 1 1 0 1 0 0 1 0 0 0 1 0 1
 0 1 1 1 1 0 0 1 1 1 1 0 0 1 0 1 1 1 1 0 1 1 1 1 1 1 1 1 1 0 0 0 1 1 1 0 1
 1 0 0 1 0 1 0 1 0 0 1 1 1 0 1 0 0 1 1 0 1 1 1 1 1 0 1 0 0 1 0 1 0 0 0 0 0
 1 0 0 1 0 0 0 0 0 0 1 1 1 1 1 0 0 0 0 0 1 0 1 1 1 1 1 0 1 0 1 0 0 0 1 1 1
 0 1 1 1 0 0 0 0 1 1 1 0 1 0 0 0 0 0 0 1 0 0 1 1 1 0 0 0 1 1 0 1 1 0 0 0 1
 0 1 1 0 1 1 1 0 1 0 1 0 1 0 0 0 1 0 0 1 1 1 0 1 0 1 1 1 1 0 1 0 1 0 1 0 0
 1 1 0 0 1 0 1 1 1 1 0 1 1 1 0 0 0 1]


In [106]:

y_test_prob = xgb_model.predict_proba(X_test_final)

y_test_pred = xgb_model.predict(X_test_final)


result_df = pd.DataFrame({'predicted_class': y_test_pred, 'probability_S': y_test_prob[:, 1], 'probability_N': y_test_prob[:, 0]})


print(result_df)



     predicted_class  probability_S  probability_N
0                  0       0.469579       0.530421
1                  1       0.896387       0.103613
2                  1       0.981489       0.018511
3                  0       0.004417       0.995583
4                  1       0.906978       0.093022
..               ...            ...            ...
235                1       0.714420       0.285580
236                0       0.104772       0.895228
237                0       0.110570       0.889430
238                0       0.377477       0.622523
239                1       0.704969       0.295031

[240 rows x 3 columns]
